In [1]:
import os
import copy
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets 
import torchvision.transforms as transforms
import pandas as pd
from tqdm.auto import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts,ExponentialLR
from sklearn.model_selection import train_test_split
from PIL import Image
import cv2
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

In [2]:
#标签和类别数进行对应
train = pd.read_csv("./kaggle/input/classify-leaves/train.csv")
labels = list(pd.read_csv("./kaggle/input/classify-leaves/train.csv")['label'])
labels_unique = list(set(list(labels))) #list index--labels
label_nums = []
for i in range(len(labels)):
    label_nums.append(labels_unique.index(labels[i]))
train['number'] = label_nums
# train.to_csv("./train_num_label.csv", index = 0) #记录对应关系

In [3]:
test = pd.read_csv("./kaggle/input/classify-leaves/test.csv")

In [4]:
train_data,eval_data = train_test_split(train,test_size=0.2,stratify=train['number'])

In [5]:
class Leaf_Dataset(Dataset):
    '''
    树叶数据集的训练集 自定义Dataset
    '''
    def __init__(self, train_csv, transform = None, test = False):
        '''
        train_path : 传入记录图像路径及其标号的csv文件
        transform : 对图像进行的变换
        '''
        super().__init__()
        self.train_csv = train_csv
        self.image_path = list(self.train_csv['image']) #图像所在地址记录
        self.test = test
        if not self.test:
            self.label_nums = list(self.train_csv['number']) #图像的标号记录
        self.transform = transform
    def __getitem__(self, idx):
        '''
        idx : 所需要获取的图像的索引
        return : image， label
        '''
        image = cv2.imread(os.path.join("./kaggle/input/classify-leaves", self.image_path[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         image = Image.open(os.path.join("/kaggle/input/classify-leaves", self.image_path[idx]))
        if(self.transform != None):
            image = self.transform(image = image)['image']
        if not self.test:
            label = self.label_nums[idx]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.image_path)

In [6]:
# #data agumentation
# transforms_train = torchvision.transforms.Compose([
#     transforms.Resize((320, 320)),
#     transforms.RandomHorizontalFlip(p=0.5),
#     transforms.RandomVerticalFlip(p=0.5),
#     transforms.ColorJitter(brightness=0.2,contrast=0.2, saturation=0.2, hue=0),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

# ])
# transforms_test = torchvision.transforms.Compose([
#     transforms.Resize((320, 320)),
#     transforms.ToTensor(),
#     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
# ])

In [7]:
transforms_train = albumentations.Compose(
    [
        albumentations.Resize(320, 320),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.Rotate(limit=180, p=0.7),
        albumentations.RandomBrightnessContrast(),
        albumentations.ShiftScaleRotate(
            shift_limit=0.25, scale_limit=0.1, rotate_limit=0
        ),
        albumentations.Normalize(
            [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
            max_pixel_value=255.0, always_apply=True
        ),
        ToTensorV2(p=1.0),
    ]
)
transforms_test = albumentations.Compose(
        [
            albumentations.Resize(320, 320),
            albumentations.Normalize(
                [0.485, 0.456, 0.406], [0.229, 0.224, 0.225],
                max_pixel_value=255.0, always_apply=True
            ),
            ToTensorV2(p=1.0)
        ]
    )

D:\Python\python 3.11.3\Lib\site-packages\albumentations\core\validation.py:87: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
C:\Users\万傲伟\AppData\Local\Temp\ipykernel_33864\3846496421.py:11: UserWarning: Argument(s) 'always_apply' are not valid for transform Normalize
  albumentations.Normalize(
C:\Users\万傲伟\AppData\Local\Temp\ipykernel_33864\3846496421.py:21: UserWarning: Argument(s) 'always_apply' are not valid for transform Normalize
  albumentations.Normalize(


In [8]:
def train_model(train_loader, valid_loader, device = torch.device("cuda:0")):
    net = torchvision.models.resnet50(weights = torchvision.models.ResNet50_Weights.IMAGENET1K_V1)
    in_features = net.fc.in_features
    net.fc = nn.Linear(in_features, 176)
    net = net.to(device)
    epoch = 30
    best_epoch = 0
    best_score = 0.0
    best_model_state = None
    early_stopping_round = 3
    losses = []
    optimizer = optim.Adam(net.parameters(), lr=0.0001,weight_decay=1e-5)
    loss = nn.CrossEntropyLoss(reduction='mean')
#     scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2, eta_min = 1e-6)
    scheduler = ExponentialLR(optimizer, gamma=0.9,verbose=True)
    for i in range(epoch):
        acc = 0
        loss_sum = 0
        net.train()
        for x, y in tqdm(train_loader):
            x = torch.as_tensor(x, dtype=torch.float)
            x = x.to(device)
            y = y.to(device)
            y_hat = net(x)
            loss_temp = loss(y_hat, y)
            loss_sum += loss_temp
            optimizer.zero_grad()
            loss_temp.backward()
            optimizer.step()
#             scheduler.step()
            acc += torch.sum(y_hat.argmax(dim=1).type(y.dtype) == y)
        scheduler.step()
        losses.append(loss_sum.cpu().detach().numpy() / len(train_loader))
        print( "epoch: ", i, "loss=", loss_sum.item(), "训练集准确度=",(acc/(len(train_loader)*train_loader.batch_size)).item(),end="")

        test_acc = 0
        net.eval()
        for x, y in tqdm(valid_loader):
            x = x.to(device)
            x = torch.as_tensor(x, dtype=torch.float)
            y = y.to(device)
            y_hat = net(x)
            test_acc += torch.sum(y_hat.argmax(dim=1).type(y.dtype) == y)
        print("验证集准确度", (test_acc / (len(valid_loader)*valid_loader.batch_size)).item())
        if test_acc > best_score:
            best_model_state = copy.deepcopy(net.state_dict())
            best_score = test_acc
            best_epoch = i
            print('best epoch save!')
        if i - best_epoch >= early_stopping_round:
            break
    net.load_state_dict(best_model_state)
    testset = Leaf_Dataset(test, transform = transforms_test,test = True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)
    device = torch.device("cuda:0")
    predictions = []
    with torch.no_grad():
        for x in tqdm(test_loader):
            x = x.to(device)
            x = torch.as_tensor(x, dtype=torch.float)
            y_hat = net(x)
            predict = torch.argmax(y_hat,dim=1).reshape(-1)
            predict = list(predict.cpu().detach().numpy())
            predictions.extend(predict)
    return predictions

In [9]:
# train_data,eval_data = train_test_split(train,test_size=0.2,stratify=train['number'])

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=2023)
prediction_df = pd.DataFrame()
for fold_n, (trn_idx, val_idx) in enumerate(skf.split(train, train['number'])):
    print(f'fold {fold_n} training...')
    train_data = train.iloc[trn_idx]
    eval_data = train.iloc[val_idx]
    trainset = Leaf_Dataset(train_data, transform = transforms_train)
    evalset = Leaf_Dataset(eval_data, transform = transforms_test)
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, drop_last=False)
    eval_loader = torch.utils.data.DataLoader(evalset, batch_size=32, shuffle=False, drop_last=False)
    predictions = train_model(train_loader, eval_loader)
    prediction_df[f'fold_{fold_n}'] = predictions

fold 0 training...


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\万傲伟/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [06:30<00:00, 262kB/s]
D:\Python\python 3.11.3\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


  0%|          | 0/459 [00:00<?, ?it/s]

In [11]:
all_predictions = list(prediction_df.mode(axis=1)[0].astype(int))
predict_label = []
for i in range(len(all_predictions)):
    predict_label.append(labels_unique[all_predictions[i]])

submission = pd.read_csv("/kaggle/input/classify-leaves/test.csv")
submission["label"] = pd.Series(predict_label)
submission.to_csv("result.csv", index=False)

In [12]:
submission

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,betula_nigra
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia
...,...,...
8795,images/27148.jpg,pinus_thunbergii
8796,images/27149.jpg,crataegus_crus-galli
8797,images/27150.jpg,pinus_thunbergii
8798,images/27151.jpg,juniperus_virginiana


In [13]:
# trainset = Leaf_Dataset(train_data, transform = transforms_train)
# evalset = Leaf_Dataset(eval_data, transform = transforms_test)
# # testset = Leaf_Test_Dataset("", transform = transforms_test)

# train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, drop_last=False)
# eval_loader = torch.utils.data.DataLoader(evalset, batch_size=32, shuffle=False, drop_last=False)
# # test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)
# net = train(train_loader, eval_loader)

In [14]:
# # TTA eval
# device = torch.device("cuda:0")
# testset = Leaf_Dataset(eval_data, transform = transforms_train)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)
# all_predictions = []
# for i in range(5):
#     predictions = []
#     with torch.no_grad():
#         for x,_ in tqdm(test_loader):
#             x = x.to(device)
#             x = torch.as_tensor(x, dtype=torch.float)
#             y_hat = net(x)
#             predict = torch.argmax(y_hat,dim=1).reshape(-1)
#             predict = list(predict.cpu().detach().numpy())
#             predictions.extend(predict)
#     all_predictions.append(predictions)
# all_predictions = list(pd.DataFrame(all_predictions).T.mode(axis=1)[0].astype(int))
# from sklearn.metrics import accuracy_score
# accuracy_score(all_predictions,eval_data['number'])

In [15]:
# testset = Leaf_Dataset(test, transform = transforms_test,test = True)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)
# device = torch.device("cuda:0")
# predictions = []
# with torch.no_grad():
#     for x in tqdm(test_loader):
#         x = x.to(device)
#         x = torch.as_tensor(x, dtype=torch.float)
#         y_hat = net(x)
#         predict = torch.argmax(y_hat,dim=1).reshape(-1)
#         predict = list(predict.cpu().detach().numpy())
#         predictions.extend(predict)

# predict_label = []
# for i in range(len(predictions)):
#     predict_label.append(labels_unique[predictions[i]])

# submission = pd.read_csv("/kaggle/input/classify-leaves/test.csv")
# submission["label"] = pd.Series(predict_label)
# submission.to_csv("result.csv", index=False)

In [16]:
# # TTA test
# device = torch.device("cuda:0")
# testset = Leaf_Dataset(test, transform = transforms_train, test=True)
# test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, drop_last=False)
# all_predictions = []
# for i in range(5):
#     predictions = []
#     with torch.no_grad():
#         for x in tqdm(test_loader):
#             x = x.to(device)
#             x = torch.as_tensor(x, dtype=torch.float)
#             y_hat = net(x)
#             predict = torch.argmax(y_hat,dim=1).reshape(-1)
#             predict = list(predict.cpu().detach().numpy())
#             predictions.extend(predict)
#     all_predictions.append(predictions)
# all_predictions = list(pd.DataFrame(all_predictions).T.mode(axis=1)[0].astype(int))
# predict_label = []
# for i in range(len(all_predictions)):
#     predict_label.append(labels_unique[all_predictions[i]])

# submission = pd.read_csv("/kaggle/input/classify-leaves/test.csv")
# submission["label"] = pd.Series(predict_label)
# submission.to_csv("result2.csv", index=False)